In [1]:
from pathlib import Path

from bertviz import head_view

import torch

In [2]:
from model.swan import SWANPhase1Only
from model.base import chkpt
from test_model import load_config, run_model_for_attention
from common.dataset import Dataset
from learner import *

In [3]:
main_path = Path('.')
data_path = main_path / 'resource'
chpt_path = main_path / 'runs_copy' / 'best_SWAN_P1'

In [4]:
tokenizer = torch.load(chpt_path / 'tokenizer.pt')
checkpoint = torch.load(chpt_path / 'SWANPhase1Only.pt')

In [5]:
config = load_config(chpt_path)
nvix = SWANPhase1Only.create_or_load(path=str(chpt_path), **config)
nvix.eval()

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predi

SWANPhase1Only(
  (encoder): TextEncoder(
    (model): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0): ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): ElectraSelfOutput(
                (dense): Linear(in_features=768, out_features=768, b

In [9]:
state_dict = nvix.state_dict()

In [10]:
print([k for k in state_dict.keys() if "explanation" in k])

['explanation._prefix_number', 'explanation.embeddings.position_ids', 'explanation.embeddings.word_embeddings.weight', 'explanation.embeddings.position_embeddings.weight', 'explanation.embeddings.token_type_embeddings.weight', 'explanation.embeddings.LayerNorm.weight', 'explanation.embeddings.LayerNorm.bias', 'explanation.encoder.layer.0.attention.self.query.weight', 'explanation.encoder.layer.0.attention.self.query.bias', 'explanation.encoder.layer.0.attention.self.key.weight', 'explanation.encoder.layer.0.attention.self.key.bias', 'explanation.encoder.layer.0.attention.self.value.weight', 'explanation.encoder.layer.0.attention.self.value.bias', 'explanation.encoder.layer.0.attention.output.dense.weight', 'explanation.encoder.layer.0.attention.output.dense.bias', 'explanation.encoder.layer.0.attention.output.LayerNorm.weight', 'explanation.encoder.layer.0.attention.output.LayerNorm.bias', 'explanation.encoder.layer.0.crossattention.self.query.weight', 'explanation.encoder.layer.0.cros

In [11]:
state_dict['explanation.encoder.layer.11.output.dense.weight'].shape

torch.Size([768, 3072])

In [12]:
state_dict['explanation.encoder.layer.11.attention.output.dense.weight'].shape

torch.Size([768, 768])

In [13]:
state_dict['explanation.encoder.layer.11.crossattention.output.dense.weight'].shape

torch.Size([768, 768])

In [12]:
state_dict['explanation.embeddings.LayerNorm.weight'].shape

torch.Size([768])

In [6]:
dataset_file = data_path / 'dataset' / 'pen.json'
test_data = Dataset(dataset_file, number_window=3)

In [7]:
test_data.select_items_with_file(data_path / 'experiments'/ 'pen' /'test')

In [8]:
test_data.num_items

365

In [9]:
set_seed(config['seed'])
batch = test_data.get_minibatches(batch_size=1, for_testing=True)
output = nvix(
    text=batch[0].text.to(nvix.device),
    beam=config['beam_for_equation'], 
    beam_expl=config['beam_for_explanation']
)

[W Context.cpp:70] Warning: torch.use_deterministic_algorithms is in beta, and its design and functionality may change in the future. (function operator())


In [26]:
nvix.explanation_pghead.attention_score[0] # Attention score: [B, T, S]

tensor([[3.3330e-01, 5.4100e-06, 1.4479e-06, 3.2128e-06, 1.9734e-07, 2.7834e-06,
         4.6896e-08, 3.4509e-07, 1.9025e-06, 2.1453e-08, 1.1091e-06, 4.9619e-05,
         7.8588e-06, 2.3247e-05, 4.8032e-06, 1.2212e-09, 3.3330e-01, 3.3330e-01]])

In [27]:
nvix.explanation_pghead.attention_score[0].shape # Attention score: [B, T, S]

torch.Size([1, 18])

In [24]:
batch[0].text.as_dict()['tokens'].shape

torch.Size([1, 18])

In [13]:
encoded, num_out = nvix.encoder(batch[0].text)
encoded.as_dict()['vector'].shape

torch.Size([1, 18, 768])

In [20]:
batch[0].as_dict()

{'text': <common.data.text.Text at 0x7f9c628b3c10>,
 'equation': Equation(operator=Label([[0, 1, 6, 3, 2]]), operands=[Label([[-1, -1, 24, 23, -1]]), Label([[-1, -1, 48, 49, -1]])]),
 'explanation': [Explanation(numbers=$[Label([[102, 1996, 2193, 1997, 7338, 6158, 2011, 7441, 102], [102, 1996, 3177, 1997, 7441, 102, -1, -1, -1]]), Label([[102, 1996, 3292, 1997, 7441, 1005, 1055, 27244, 2075, 1999, 7338, 102, -1, -1], [102, 1996, 3177, 1997, 7441, 1005, 1055, 27244, 2075, 1999, 7338, 2566, 3178, 102]]), Label([[102, 1996, 2561, 2193, 1997, 7338, 20354, 102, -1], [102, 1996, 2193, 1997, 7338, 20354, 2566, 3178, 102]])], variables=$[Label([[102, 1996, 2193, 1997, 2847, 6158, 2011, 7441, 102]]), Label([[102, 1996, 3091, 1997, 7441, 1005, 1055, 27244, 2075, 2051, 102]]), Label([[102, 1996, 2193, 1997, 2847, 20354, 102]])], worker=$-1)],
 'info': [<common.data.ExtraInfo at 0x7f9c42fa57c0>]}

In [30]:
output

{'eqn_ignore': {1},
 'var_expl': [Label([[102, 1996, 3091, 1997, 2051, 1997, 27244, 2075, 102, -1, -1, -1, -1]])],
 'num_expl': [Label([[102, 1996, 3292, 1997, 27244, 2075, 1999, 7338, 102, -1, -1], [102, 1996, 3177, 1997, 27244, 2075, 1999, 7338, 2566, 3178, 102]])],
 'explanation': [Explanation(numbers=$[Label([[102, 1996, 3292, 1997, 27244, 2075, 1999, 7338, 102, -1, -1], [102, 1996, 3177, 1997, 27244, 2075, 1999, 7338, 2566, 3178, 102]])], variables=$[Label([[102, 1996, 3091, 1997, 2051, 1997, 27244, 2075, 102, -1, -1, -1, -1]])], worker=$0)],
 'equation': Equation(operator=Label([[0]]), operands=[Label([[-1]]), Label([[-1]])])}

In [35]:
output['num_expl'][0].as_dict()['indices'].shape

torch.Size([2, 11])

In [38]:
import numpy
import matplotlib.pyplot as plt

# http://stackoverflow.com/questions/14391959/heatmap-in-matplotlib-with-pcolor
def plot_head_map(mma, target_labels, source_labels):
    fig, ax = plt.subplots()
    heatmap = ax.pcolor(mma, cmap=plt.cm.Blues)
    # put the major ticks at the middle of each cell
    ax.set_xticks(numpy.arange(mma.shape[1]) + 0.5, minor=False) # mma.shape[1] = target seq 길이
    ax.set_yticks(numpy.arange(mma.shape[0]) + 0.5, minor=False) # mma.shape[0] = input seq 길이
 
    # without this I get some extra columns rows
    # http://stackoverflow.com/questions/31601351/why-does-this-matplotlib-heatmap-have-an-extra-blank-column
    ax.set_xlim(0, int(mma.shape[1]))
    ax.set_ylim(0, int(mma.shape[0]))
 
    # want a more natural, table-like display
    ax.invert_yaxis()
    ax.xaxis.tick_top()
 
    # source words -> column labels
    ax.set_xticklabels(source_labels, minor=False)
    # target words -> row labels
    ax.set_yticklabels(target_labels, minor=False)
 
    plt.xticks(rotation=45)
 
    # plt.tight_layout()
    plt.show()

    
def read_plot_alignment_matrices(source_labels, target_labels, alpha):
 
    mma = alpha.cpu().data.numpy()
 
    plot_head_map(mma, target_labels, source_labels)

In [48]:
attn = state_dict['explanation.encoder.layer.11.output.LayerNorm.weight']
attn.shape
head_view(attn, encoded_input)

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.

In [29]:
input_str = "the sears tower in chicago is 145 ##0 feet tall . the john hancock center in chicago is 112 ##7 feet tall . suppose you are asked to build a small - scale replica of each . if you make the sears tower 3 meter tall , what would be the approximate height of the john hancock replica ? round your answer to the nearest hundred ##th ."
encoded_input = tokenizer.encode(input_str)
len(encoded_input)

76

In [32]:
output_str = "the height of the sears tower"
encoded_output = tokenizer.encode(output_str)
len(encoded_output)

8

In [27]:
config

{'seed': 1,
 'batch_size': 16,
 'beam_for_equation': 3,
 'beam_for_explanation': 5,
 'dataset': '/home/bydelta/SimpleFESTA/resource/dataset/pen.json',
 'learner': {'model': 'SWAN_P1',
  'encoder': 'google/electra-base-discriminator',
  'equation': {'hidden_dim': 0, 'intermediate_dim': 0, 'layer': 6, 'head': 0},
  'explanation': {'encoder': 'google/electra-base-discriminator',
   'shuffle': True}},
 'resource': {'num_gpus': 1.0, 'num_cpus': 1.0},
 'experiment': {'dev': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/dev',
   'period': 100},
  'train': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/train'},
  'test': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/test',
   'period': 500}},
 'grad_clip': 10.0,
 'optimizer': {'type': 'lamb',
  'lr': 0.00176,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'debias': True},
 'scheduler': {'type': 'warmup-linear',
  'num_warmup_epochs': 10.0,
  'num_total_epochs': 500},
 'window': 3}

In [7]:
dataset_file = data_path / 'dataset' / 'pen.json'

KEY_DATASET = 'dataset'
KEY_SEED = 'seed'

exp_base = {
        KEY_DATASET: str(dataset_file.absolute()),
        KEY_SEED: config['seed']
    }

In [ ]:
test_check, test_results = run_model_for_attention(chpt_path, **exp_base)

In [ ]:
test_results